<a name="top"> <h1>01. ML Models</h1> <a>

<p>Análisis de sentimiento: Tweets<br />
<strong>Trabajo de Fin de Master</strong><br />
<strong>Master Universitario en Ciencia de Datos</strong></p>


<p style="text-align:right">V&iacute;ctor Viloria V&aacute;zquez (<em>victor.viloria@cunef.edu</em>)</p>


<hr style="border:1px solid gray">

### Estructura

[Introducción](#introduccion) 

[1. Librerias utilizadas y funciones](#librerias) 

[2. Lectura del dataframe y preparación de los datos](#lectura) 

   - 2.1. Lectura del DF
   - 2.2. Preparación de los datos

[3. Evaluación de los modelos ](#modelos) 

   - 3.1. Support Vector Classifier
   - 3.2. Multinomial Naive Bayes
   - 3.3. Random Forest
   - 3.4. Logistic Regression


[4. Resultado de los modelos](#evaluacion) 


[5. Calibración del modelo](#calibracion)


<hr style="border:1px solid gray">

# <a name="introduccion"> Introducción <a>


En este segundo notebook procederemos a desarrollar diferentes modelos de Machine Learning en base al dataframe que preprocesamos en `0_DataPreprocessing`.

El objetivo será probar diferentes modelos de Machine Learning para poder clasificarlos según su rendimiento y observar si en función del texto, la puntuación asignada sería acorde a la del usuario.

Para el desarrollo de estas tareas utilizaremos la librería de SKLearn, la cual dispone de diferentes modelos de clasificación que funcionan bastante bien. En particular, probaremos varios modelos, como el SVC (Support Vector Classification), el Multinomial Naive Bayes, Random Forest y Regresión Logística.

# <a name="librerias"> 1. Librerias utilizadas y funciones <a>


Importamos las librerias a utilizar para la creación de modelos de ML:

In [1]:
# Import basic libraries.

import pandas as pd
import numpy as np
import string

# Import ML libraries.

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Import libraries for evaluation.

from sklearn.model_selection import GridSearchCV

In [27]:
# Import basic libraries.

import pandas as pd
import matplotlib.pyplot as plt
import pickle 
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


# Import sklearn metrics.

from sklearn.metrics import  fbeta_score, roc_curve, classification_report,accuracy_score,roc_auc_score


# <a name="lectura"> 2. Lectura del dataframe y preparación de los datos<a>


## 2.1. Lectura del DF

In [2]:
#Import parquet file.

reviews = pd.read_parquet('../../data/processed/reviews.parquet')

# Show the head of the dataframe.

reviews.head()

,reviewText,overall
0,amazon shopping amazoncom gift cards christmas...,1
1,got gift card friend best site much choose gre...,5
2,arent going save trees people complaining pape...,5
3,always get someone something amazon safety net...,5
4,take 50 dollars good money limitations turn am...,1


## 2.2. Preparación de los datos

Preparamos los datos para que puedan ser introducidos dentro de los modelos, separando en X el texto y en Y las puntuaciones.

In [3]:
# Divide the data into X and y.

X = reviews.reviewText
y = reviews.overall

# Split into train and test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# <a name="modelos"> 3. Evaluación de los modelos<a>


## 3.1. Support Vector Classifier

In [4]:

tfidf = TfidfVectorizer(stop_words="english",max_df=0.99,min_df=0.01)
svc = SVC(probability=True)
model = make_pipeline(tfidf, svc)
#Training
model.fit(x_train, y_train)

#Prediccion sobre test
preds= model.predict(x_test)
predict_probabilities = model.predict_proba(x_test)


In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


# Use classification report to evaluate the model.

classification_report = classification_report(y_test, preds)


print("Classification Report:\n", classification_report)

Classification Report:
               precision    recall  f1-score   support

           1       0.61      0.32      0.42      1420
           2       1.00      0.00      0.01       316
           3       0.00      0.00      0.00       597
           4       0.00      0.00      0.00      1739
           5       0.88      0.99      0.93     25367

    accuracy                           0.87     29439
   macro avg       0.50      0.26      0.27     29439
weighted avg       0.80      0.87      0.82     29439



c:\Users\vvict\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vvict\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vvict\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# List with the titles and parameters

titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]

# Plot the confusion matrix

for title, normalize in titles_options:
    disp = plot_confusion_matrix(model, x_test, y_test,
                                 # display_labels=ytest,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

In [16]:
x_train

739       perfect bridal shower perfect size weddings sh...
51861     love gift cards little amazon boxes cute howev...
85322     love pizza got great value lightning deal sale...
56676                            nice got fast good quality
47567     great gift loved ones live far away saves stam...
                                ...                        
41239     live country amazon gift cards way go gifts gi...
55985     gift lifesaver holiday season great gift perso...
32399     sure give someone give amazon gift card get wh...
82584     birthday gift card perfect present kindle read...
121909                                   much say gift card
Name: reviewText, Length: 117755, dtype: object

In [14]:
preds

array([5, 5, 5, ..., 5, 5, 5], dtype=int64)

In [8]:
model.predict_proba(x_test)

AttributeError: predict_proba is not available when  probability=False

In [7]:
np.mean(preds==y_test)

0.8723122388668093

## 3.2. Multinomial Naive Bayes

In [8]:
# Entrenar el modelo
clf = MultinomialNB()

model = make_pipeline(tfidf, clf)
#Training
model.fit(x_train, y_train)

#Prediccion sobre test
preds= model.predict(x_test )

In [9]:
# Check the accuracy.

np.mean(preds==y_test)

0.8631407316824621

## 3.3. Random Forest

In [11]:
# Entrenar el modelo

rf = RandomForestClassifier()

model = make_pipeline(tfidf, rf)

#Training

model.fit(x_train, y_train)

#Prediccion sobre test

preds= model.predict(x_test )

# Evaluate the model

np.mean(preds==y_test)

0.8662318692890384

## 3.4. Logistic Regression

In [12]:
# Entrenar el modelo

lr = LogisticRegression()

model = make_pipeline(tfidf, lr)

#Training

model.fit(x_train, y_train)

#Prediccion sobre test

preds= model.predict(x_test )

# Evaluate the model

np.mean(preds==y_test)

0.8713271510581202

# <a name="evaluacion"> 4. Resultado de los modelos <a>


Observamos los restultados arrojados por los modelos:

In [7]:
# Create a dataframe with the results of each model.

results = pd.DataFrame({'Model': ['SVC', 'MultinomialNB', 'RandomForestClassifier', 'LogisticRegression'], 'Accuracy': [0.872, 0.863, 0.866, 0.871]})

# Show the results.

results.sort_values(by='Accuracy', ascending=False).reset_index(drop=True)

,Model,Accuracy
0,SVC,0.872
1,LogisticRegression,0.871
2,RandomForestClassifier,0.866
3,MultinomialNB,0.863


# <a name="calibracion"> 5. Calibración del modelo <a>


In [ ]:
# Tune the hyperparameters of the SVC model.

parameters = {'svc__C': [0.1, 1, 10], 'svc__gamma': [0.001, 0.01, 0.1]}
grid = GridSearchCV(model, parameters, cv=5)
grid.fit(x_train, y_train)

# Show the best parameters.

grid.best_params_

# Show the best score.

grid.best_score_
